TFLite Model Maker Tensorflow Documentation: https://www.tensorflow.org/lite/models/modify/model_maker/image_classification

### Importing Libraries

In [7]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

In [8]:
tf.__version__  #2.4.1

'2.4.1'

In [9]:
tf.test.is_gpu_available() # gpu is avaliable or not

True

### Train set path

In [10]:
image_path = os.path.join(os.path.dirname("/home/ubuntu/FatihNewModel/Datasets/TFLite_56-44/train"))
#/home/ubuntu/FatihNewModel/Datasets/TFLite_56-44/train = serverdaki train set 
#localde calisiliyorsa train set'in bulundugu path girilmeli

### Train-Test Split 

In [11]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 40671, num_label: 2, labels: fincan, red.


INFO:tensorflow:Load image with size: 40671, num_label: 2, labels: fincan, red.


### Train, test and export the model

#### Simple version:

In [12]:
# Create the model.
model = image_classifier.create(train_data)

# Evaluate the model.
loss, accuracy = model.evaluate(test_data)

# Export to Tensorflow Lite model and label file in `export_dir`.
model.export(export_dir='./', tflite_filename='model.tflite')

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5
1143/1143 [==============================] - 173s 104ms/step - loss: 0.2471 - accuracy: 0.9778
Epoch 2/5
1143/1143 [==============================] - 119s 104ms/step - loss: 0.2173 - accuracy: 0.9959
Epoch 3/5
1143/1143 [==============================] - 119s 104ms/step - loss: 0.2161 - accuracy: 0.9964
Epoch 4/5
1

INFO:tensorflow:Assets written to: /tmp/tmpto8q09d4/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp7ev94kqj/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmp7ev94kqj/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model3_new.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model3_new.tflite


#### Advanced and detailed version:

modelspec = transfer learning yapilacak modelin(feature vector) linki girilir. secilen feature vector ne kadar buyukse model maker'in export ettigi model de o kadar buyuk olur. 
https://tfhub.dev/s?module-type=image-feature-vector&tf-version=tf2 linkinden farkli feature vectorler secilebilir.


modelspec.input_image_shape = [224,224] koduna secilen feature vector'un input degerleri girilir.

2 farkli quantization yontemi var. dynamic ve float16. ikisinin modeli kucultme oranlari ve performans degerleri farkli sonuclar veriyor. model quantize edilmek istenmiyorsa ,quantization_config=config kismi cikartilmalidir.

In [13]:
modelspec = image_classifier.ModelSpec(
    uri='https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5')
##modelspec.input_image_shape = [xxx,xxx] xxx field'ina secilen vektorun dokumantasyonunda yazan boyut girilir.
modelspec.input_image_shape = [224,224]

model = image_classifier.create(train_data, model_spec=modelspec, epochs=5, learning_rate=0.013,
                                train_whole_model=True, shuffle=False, use_augmentation=False)

# Evaluate the model.
loss, accuracy = model.evaluate(test_data)

# Export to Tensorflow Lite model and label file in `export_dir`.
config = QuantizationConfig.for_dynamic()
#config = QuantizationConfig.for_float16()
model.export(export_dir='./', tflite_filename='model.tflite', quantization_config=config)
#model.export(export_dir='./', tflite_filename='model.tflite')

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_2 (HubK (None, 1024)              1026552   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 1,028,602
Trainable params: 1,018,922
Non-trainable params: 9,680
_________________________________________________________________
None
Epoch 1/5
1143/1143 [==============================] - 131s 110ms/step - loss: 0.4861 - accuracy: 0.9787
Epoch 2/5
1143/1143 [==============================] - 126s 110ms/step - loss: 0.4464 - accuracy: 0.9984
Epoch 3/5
1143/1143 [==============================] - 126s 110ms/step - loss: 0.4401 - accuracy: 0.9998
Epoch 4/5
1143/1143 [=============================

INFO:tensorflow:Assets written to: /tmp/tmpci9abz84/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp25qo_q3g/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmp25qo_q3g/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model840_new.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model840_new.tflite
